In [0]:
import nltk
import random
import math
import re
import string
from nltk.util import bigrams
from nltk.util import ngrams
from nltk import FreqDist
from nltk.util import everygrams
from nltk.lm import MLE
from nltk.tokenize import sent_tokenize
#nltk.download('punkt')
from sklearn.model_selection import train_test_split

In [0]:
#file reading
f=open("trump.txt","r")
content=f.read()

In [0]:
#data preprocessing
content=content.replace('\n','')
content=content.lower()

In [0]:
#tokenization
sentences=sent_tokenize(content)
sentences=['<s> '+re.sub(r'[^\w\s]','',s)+' </s>' for s in sentences]

In [0]:
#building a corpus
random.shuffle(sentences)
train_data=sentences[:math.ceil(0.8*len(sentences))]
test_data=sentences[math.ceil(0.8*len(sentences)):]
words=[]
train_words=[]
test_words=[]
for i in sentences:
    for j in i.split():
        words.append(j)

for i in train_data:
    for j in i.split():
        train_words.append(j)

for i in test_data:
    for j in i.split():
        test_words.append(j)

        

In [0]:
#generating a unigram,bigram,trigram,quadgram
unigrams=ngrams(train_words,1)
bigrams=ngrams(train_words,2)
trigrams=ngrams(train_words,3)
quadgrams=ngrams(train_words,4)

In [0]:
#creating a dictionary for unigrams,bigrams,trigrams,quadgrams
def calc(ngrams,n):
    freq=FreqDist(ngrams)
    count={}
    l=0
    for k,v in freq.items():
        #print(k,v)
        s=""
        for i in range(n):
           s=s+k[i]+" "
        count[s]=v
        l=l+v
    return count

In [0]:
unigramcount=calc(unigrams,1)
bigramcount=calc(bigrams,2)
trigramcount=calc(trigrams,3)
quadgramcount=calc(quadgrams,4)

In [0]:
print("number of unigrams ",sum(unigramcount.values()))
print("number of bigrams ",sum(bigramcount.values()))
print("number of trigrams ",sum(trigramcount.values()))
print("number of quadgrams ",sum(quadgramcount.values()))

number of unigrams  149881
number of bigrams  149880
number of trigrams  149879
number of quadgrams  149878


In [0]:
#creating a dictionary which maintains a probability for n-grams
def calc_prob(N,V,upper,n):
    probdict={}
    if n==1:
        for w in upper:
            probdict[w]=(upper.get(w,0.01)+1)*1.0/(N+V)
    else:
        for w in upper:
            s=" ".join(w.split()[:n-1])+" "
            probdict[w]=(upper[w])*1.0/(N[s]+V)
    return probdict

In [0]:
unigram_prob=calc_prob(len(train_words),len(unigramcount),unigramcount,1)
bigram_prob=calc_prob(unigramcount,len(unigramcount),bigramcount,2)
trigram_prob=calc_prob(bigramcount,len(unigramcount),trigramcount,3)
quadgram_prob=calc_prob(trigramcount,len(unigramcount),quadgramcount,4)

In [0]:
##function to calculate perplexity for n-grams
def perplexity(test_token,upper,lower,n,N):
    #test_token=test_token.split()
    per=float(1.0)
    if n==1:
        for i in test_token:
            prob=upper.get(i+" ",0.00001)*1.0/lower
            per=per* (prob**(1./N))
    else:
        for i in range(len(test_token)-n):
            den=" ".join(test_token[i:i+n-1])+" "
            num=" ".join(test_token[i:i+n])+" "
            prob=upper.get(num,0.00001)*1.0/lower.get(den,0.00001)
            per=per* (prob**(1./N))
    return per

In [0]:
print("unigram ",perplexity(test_words,unigramcount,len(train_words),1,len(train_words)))
print("bigram ", perplexity(test_words,bigramcount,unigramcount,2,len(train_words)))
print("trigram ",perplexity(test_words,trigramcount,bigramcount,3,len(train_words)))
print("quadgram ",perplexity(test_words,quadgramcount,trigramcount,4,len(train_words)))

unigram  0.20632755575716172
bigram  0.25028552471185506
trigram  0.30442822132281905
quadgram  0.4577999688102268


In [0]:
#function to generate a random sentence using unigrams
def unigram_generate(unigramcount,word):
    for j in range(30):
            d=random.random()
            result=float(0.0)
            for i in unigramcount.keys():
                if i.strip()=="<s>":
                    continue
                result=result+unigramcount[i]
                if result>=d:
                    if i.strip()=="</s>":
                        print('.',end=' ')
                        return
                    print(i.strip(),end=' ')
                    break
    
        

In [0]:
for i in range(5):
    unigram_generate(unigram_prob,'<s>')

the deductions on the instruction korea want do wall has having poll you . saying get government no announced . . have ford but has very mexico and policy to to laws podiumtom hope going companies know finestso and know settled were from way build principals . eight . 

In [0]:
#function to generate a random sentence using bigrams
def bigram_generate(bigramprob,word):
    for j in range(30):
        d=random.random()
        x=float(0.0)
        for i in bigramprob.keys():
            token=i.split()
            if token[0]==word.strip():
                x+=bigramprob[i]
        if d>x:
            d=x*d
        result=float(0.0)
        for i in bigramprob.keys():
            token=i.split()
            if token[1]=='<s>':
                continue
            if token[0]==word:
                result+=bigramprob[i]
                if result>=d:
                    word=token[1]
                    if token[1]=='</s>':
                        print('.',end=' ')
                        return
                    print(token[1],end=' ')
                    break
                    
        
        
    
    

In [0]:
for i in range(5):
    bigram_generate(bigram_prob,'<s>')

i guess . thats the job . i mean where im leading everything after another one of afghan parents who is that work all for political phenomenon . but millions of papers that . theyll be nice to stage with hillary clinton explained that when you know . 

In [0]:
#function to generate a random sentence using trigrams
def trigram_generate(trigramprob,bigramprob,word):
    
    for j in range(50):
        d=random.random()
        x=float(0.0)
        #print("word ",word)
        #print(" d ",d)    
        if j==0:
            for i in bigramprob.keys():
                token=i.split()
                #print(token)
                if token[0]==word:
                    x+=bigramprob[i]
            if d>x:
                d=x*d
            result=float(0.0)
            for i in bigramprob.keys():
                token=i.split()
                if token[1]=='<s>':
                    continue
                if token[0]==word:
                    result+=bigramprob[i]
                    if result>=d:
                        word=i
                        if token[1]=='</s>':
                            print('.',end=' ')
                            return
                        else:
                            print(token[1],end=' ')
                        break
        else:
            for i in trigramprob.keys():
                token=i.split()
                w=word.split()
                if token[0]==w[0] and token[1]==w[1]:
                    x+=trigramprob[i]
                if d>x:
                    d=x*d
                result=float(0.0)
            #print("after d",d)
            for i in trigramprob.keys():
                token=i.split()
                w=word.split() #print("result ",result)
                if token[2]=='<s>':
                    continue
                if token[0]==w[0] and token[1]==w[1]:
                    result+=trigramprob[i]
                    if result>=d:
                        word=token[1]+' '+token[2]
                        if token[2]=='</s>':
                            print('.',end=' ')
                            return
                        else:
                            print(token[2],end=' ')
                        break
            
            
    

In [0]:
for i in range(5):
    trigram_generate(trigram_prob,bigram_prob,'<s>')

believe me you know shes been involved in corruption her whole life whether its whitewater . he borrowed a million dollars at an interest rate that everybody in this room would be proud to have a disaster called the big lie obamacare . who is better than us in education and many other things . they cant make a deal because they have no leaderits not like oh gee we disagree on obamacare . everybody knows its a five year statute of limitations . 

In [0]:
#function to generate a random sentence using quadgrams
def quadgram_generate(quadgramprob,trigramprob,bigramprob,word):
    
    for j in range(50):
        d=random.random()*0.00001
        x=float(0.0)
        #print("word ",word)
        #print(" d ",d)    
        if j==0:
            for i in bigramprob.keys():
                token=i.split()
                #print(token)
                if token[0]==word:
                    x+=bigramprob[i]
            if d>x:
                d=x*d
            result=float(0.0)
            for i in bigramprob.keys():
                token=i.split()
                if token[1]=='<s>':
                    continue
                if token[0]==word:
                    result+=bigramprob[i]
                    if result>=d:
                        word=i
                        if token[1]=='</s>':
                            print('.',end=' ')
                            return
                        else:
                            print(token[1],end=' ')
                        break
        elif j==1:
            for i in trigramprob.keys():
                token=i.split()
                w=word.split()
                if token[0]==w[0] and token[1]==w[1]:
                    x+=trigramprob[i]
                if d>x:
                    d=x*d
                result=float(0.0)
            #print("after d",d)
            for i in trigramprob.keys():
                token=i.split()
                w=word.split() #print("result ",result)
                if token[2]=='<s>':
                    continue
                if token[0]==w[0] and token[1]==w[1]:
                    result+=trigramprob[i]
                    if result>=d:
                        word=i
                        if token[2]=='</s>':
                            print('.',end=' ')
                            return
                        else:
                            print(token[2],end=' ')
                        break
        else:
            for i in quadgramprob.keys():
                token=i.split()
                w=word.split()
                if token[0]==w[0] and token[1]==w[1] and token[2]==w[2]:
                    x+=quadgramprob[i]
                if d>x:
                    d=x*d
                result=float(0.0)
            #print("after d",d)
            for i in quadgramprob.keys():
                token=i.split()
                w=word.split() #print("result ",result)
                if token[3]=='<s>':
                    continue
                if token[0]==w[0] and token[1]==w[1] and token[2]==w[2]:
                    result+=quadgramprob[i]
                    if result>=d:
                        word=' '.join(token[:3])+' '
                        if token[3]=='</s>':
                            print('.',end=' ')
                            return
                        else:
                            print(token[3],end=' ')
                        break
            
            
    

In [0]:
for i in range(5):
    quadgram_generate(quadgram_prob,trigram_prob,bigram_prob,'<s>')

no they want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want no they want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want no they want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want no they want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want want no they 